# Chapter 13: Getting Started with Kubernetes

Having established the architectural foundations of Kubernetes in Chapter 12, we now transition from theoretical understanding to practical implementation. Before deploying production workloads, you need a local development environment that accurately mirrors production behavior while remaining lightweight enough for iterative development. This chapter guides you through installing local Kubernetes clusters, configuring the kubectl command-line interface, and executing your first deployments.

Whether you are developing on macOS, Windows, or Linux, modern tooling provides Kubernetes clusters that run directly on your workstation with minimal resource overhead. We will explore the trade-offs between Minikube, Kind, and k3d to help you select the appropriate tool for your specific development workflow.

## 13.1 Local Kubernetes Options

Local Kubernetes clusters serve multiple purposes: learning the platform, developing applications against realistic APIs, and testing configurations before production deployment. Unlike production clusters with dedicated control plane nodes, local solutions package everything into a single executable or container.

### Option 1: Minikube

Minikube is the original local Kubernetes implementation and remains the most widely supported option. It creates a single-node cluster inside a virtual machine (VM) or container, supporting multiple virtualization backends.

**Architecture:**
Minikube provisions a VM (using Hyper-V, VirtualBox, VMware, or KVM) or Docker container containing:
- Single-node Kubernetes cluster (control plane and worker combined)
- Container runtime (containerd, CRI-O, or Docker)
- Default storage class and networking plugins
- Addon manager for optional components (Ingress, Dashboard, Registry)

**Installation:**

macOS (using Homebrew):
```bash
brew install minikube
```

Windows (using Chocolatey):
```powershell
choco install minikube
```

Linux:
```bash
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
```

**Starting the Cluster:**

```bash
# Start with default settings (Docker driver on Linux/macOS, Hyper-V on Windows)
minikube start

# Start with specific Kubernetes version matching production
minikube start --kubernetes-version=v1.28.3

# Start with increased resources for resource-intensive applications
minikube start --cpus=4 --memory=8192 --disk-size=50g

# Start with specific container runtime
minikube start --container-runtime=containerd

# Start with CNI plugin for network policy testing
minikube start --cni=calico
```

**Key Commands:**
```bash
# Check cluster status
minikube status

# Open Kubernetes Dashboard (built-in)
minikube dashboard

# Access cluster services via tunnel
minikube service my-service

# SSH into the node for debugging
minikube ssh

# Pause cluster without deleting (preserve state)
minikube pause

# Stop cluster
minikube stop

# Delete cluster and clean up
minikube delete
```

**Add-ons:**
Minikube provides pre-configured add-ons for common requirements:
```bash
# Enable Ingress controller (NGINX)
minikube addons enable ingress

# Enable local registry
minikube addons enable registry

# Enable metrics-server for kubectl top
minikube addons enable metrics-server

# List available addons
minikube addons list
```

**When to Use Minikube:**
- You need a VM-isolated environment separate from your Docker installation
- You require specific Kubernetes versions for compatibility testing
- You need persistent volumes that survive cluster restarts
- You are learning Kubernetes and want the most documented solution

### Option 2: Kind (Kubernetes IN Docker)

Kind runs Kubernetes clusters using Docker containers as nodes. Unlike Minikube's VM approach, Kind creates "node" containers that run systemd and Kubernetes components, making it significantly faster to start and more resource-efficient.

**Architecture:**
- Control plane and worker nodes are Docker containers
- Uses kubeadm for cluster provisioning (production-like setup)
- Supports multi-node clusters (including HA configurations) on a single machine
- Ideal for CI/CD pipelines due to speed and Docker-native approach

**Installation:**

```bash
# macOS/Linux
curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.20.0/kind-$(uname)-amd64
chmod +x ./kind
sudo mv ./kind /usr/local/bin/kind

# Or via Homebrew
brew install kind
```

**Creating Clusters:**

```bash
# Create default single-node cluster
kind create cluster

# Create named cluster (allows multiple clusters)
kind create cluster --name dev-cluster

# Create multi-node cluster for testing pod distribution
cat <<EOF | kind create cluster --config=-
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
- role: worker
- role: worker
- role: worker
EOF

# Create cluster with specific Kubernetes version
kind create cluster --image kindest/node:v1.28.0

# Create cluster with port mappings for local development
cat <<EOF | kind create cluster --config=-
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
  extraPortMappings:
  - containerPort: 80
    hostPort: 8080
    listenAddress: "0.0.0.0"
    protocol: TCP
  - containerPort: 443
    hostPort: 8443
    listenAddress: "0.0.0.0"
    protocol: TCP
EOF
```

**Loading Images:**
A key advantage of Kind is loading local images without registries:
```bash
# Build image locally
docker build -t my-app:latest .

# Load into Kind cluster
kind load docker-image my-app:latest --name dev-cluster

# Or load from archive
kind load image-archive my-app.tar --name dev-cluster
```

**Key Commands:**
```bash
# List clusters
kind get clusters

# Delete cluster
kind delete cluster --name dev-cluster

# Export kubeconfig for specific cluster
kind export kubeconfig --name dev-cluster

# Get cluster information
kubectl cluster-info --context kind-dev-cluster
```

**When to Use Kind:**
- You need multi-node clusters for testing scheduling, affinity, or distributed systems
- You are running CI/CD pipelines (fast startup, easy cleanup)
- You want to test Kubernetes itself or controllers
- You prefer Docker-native workflows without VM overhead

### Option 3: k3d (K3s in Docker)

k3d is a lightweight wrapper to run k3s (Rancher's minimal Kubernetes distribution) in Docker. K3s is a CNCF-certified Kubernetes distribution designed for resource-constrained environments, stripping out legacy features and using SQLite instead of etcd (by default).

**Architecture:**
- Single binary Kubernetes (control plane and kubelet combined)
- Uses SQLite for storage instead of etcd (lighter weight)
- Ships with Traefik ingress controller and local-path-provisioner by default
- Extremely fast startup (seconds vs. minutes)

**Installation:**

```bash
# macOS (Homebrew)
brew install k3d

# Linux
curl -s https://raw.githubusercontent.com/k3d-io/k3d/main/install.sh | bash

# Windows (Chocolatey)
choco install k3d
```

**Creating Clusters:**

```bash
# Create simple cluster
k3d cluster create mycluster

# Create with specific agents (workers) and servers (control plane)
k3d cluster create mycluster --servers 1 --agents 3

# Create with port mapping and registry
k3d cluster create mycluster \
  --port "8080:80@loadbalancer" \
  --port "8443:443@loadbalancer" \
  --registry-create mycluster-registry

# Create with volume mounts for local development
k3d cluster create mycluster \
  --volume /home/user/code:/code@all \
  --volume /home/user/data:/data@agent:0,1,2
```

**Registry Integration:**
k3d excels at local registry management:
```bash
# Create cluster with registry
k3d cluster create mycluster --registry-create myregistry

# Tag image for k3d registry
docker tag myapp:latest k3d-myregistry:5000/myapp:latest

# Push to local registry
docker push k3d-myregistry:5000/myapp:latest

# Use in deployment
# image: k3d-myregistry:5000/myapp:latest
```

**Key Commands:**
```bash
# List clusters
k3d cluster list

# Start/Stop cluster (preserve state)
k3d cluster stop mycluster
k3d cluster start mycluster

# Delete cluster
k3d cluster delete mycluster

# Get kubeconfig
k3d kubeconfig get mycluster > ~/.kube/config
k3d kubeconfig merge mycluster --kubeconfig-switch-context

# List nodes
k3d node list
```

**When to Use k3d:**
- You need the fastest possible startup time
- You are developing on resource-constrained machines (laptops with limited RAM)
- You want built-in Ingress and registry support without manual addon installation
- You prefer the simplicity of k3s architecture

### Comparison Matrix

| Feature | Minikube | Kind | k3d |
|---------|----------|------|-----|
| **Startup Time** | 2-5 minutes | 30-60 seconds | 10-20 seconds |
| **Resource Usage** | High (VM overhead) | Medium (Docker) | Low (Docker + lightweight K8s) |
| **Multi-node Support** | Limited | Excellent | Good |
| **Kubernetes Version Selection** | Any version | Specific images | Specific images |
| **VM Required** | Yes (Linux optional) | No | No |
| **Built-in Ingress** | Via addon | Manual setup | Yes (Traefik) |
| **Built-in Registry** | Via addon | Manual setup | Yes (optional) |
| **Best For** | Beginners, specific K8s versions | CI/CD, multi-node testing | Rapid development, low resources |

### Installation Verification

Regardless of your choice, verify functionality:

```bash
# Check cluster info
kubectl cluster-info

# View nodes
kubectl get nodes -o wide

# Check system pods
kubectl get pods -n kube-system

# Test DNS resolution
kubectl run -it --rm debug --image=busybox:1.36 --restart=Never -- nslookup kubernetes.default
```

## 13.2 Cloud Kubernetes Options

While local clusters serve development needs, understanding cloud-managed options helps you prepare for production. Cloud providers offer Kubernetes control plane management, eliminating operational overhead.

**Amazon EKS (Elastic Kubernetes Service):**
- Managed control plane across multiple AZs
- Integrated with AWS IAM for authentication
- Fargate option for serverless containers
- VPC networking with native integration

**Google GKE (Google Kubernetes Engine):**
- Autopilot mode (Google manages nodes) or Standard mode
- Auto-repair and auto-scaling built-in
- Integration with Google Cloud Load Balancing
- Anthos for multi-cloud deployments

**Azure AKS (Azure Kubernetes Service):**
- Integrated Azure Active Directory
- Virtual node option using Azure Container Instances
- GitOps integration with Flux
- Azure Policy enforcement

**Selection Criteria:**
- **Ecosystem proximity**: Choose the cloud where your data resides
- **Pricing model**: Control plane fees ($0.10/hour for EKS/AKS, GKE free tier available)
- **Integration needs**: IAM, networking, and storage integration requirements
- **Compliance**: Specific certifications required (FedRAMP, HIPAA, PCI DSS)

For initial learning, start with local clusters; migrate to cloud-managed solutions when moving toward production or requiring persistent infrastructure.

## 13.3 Installing kubectl

kubectl is the command-line interface for interacting with Kubernetes clusters. It communicates with the API server to deploy applications, inspect resources, and manage cluster operations.

### Installation Methods

**macOS:**

```bash
# Using Homebrew (recommended)
brew install kubectl

# Or using curl
curl -LO "https://dl.k8s/release/$(curl -L -s https://dl.k8s/release/stable.txt)/bin/darwin/amd64/kubectl"
chmod +x ./kubectl
sudo mv ./kubectl /usr/local/bin/kubectl
```

**Linux:**

```bash
# Using native package managers (Ubuntu/Debian)
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates curl
curl -fsSL https://pkgs.k8s.io/core:/stable:/v1.28/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg
echo 'deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v1.28/deb/ /' | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl

# Or using snap
sudo snap install kubectl --classic

# Or binary download
curl -LO "https://dl.k8s/release/$(curl -L -s https://dl.k8s/release/stable.txt)/bin/linux/amd64/kubectl"
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
```

**Windows:**

```powershell
# Using Chocolatey
choco install kubernetes-cli

# Using winget
winget install -e --id Kubernetes.kubectl

# Or curl
curl.exe -LO "https://dl.k8s/release/v1.28.0/bin/windows/amd64/kubectl.exe"
# Add to PATH manually or move to C:\Windows\System32
```

### Verification

```bash
# Verify installation
kubectl version --client

# Output should show:
# Client Version: v1.28.x
# Kustomize Version: v5.x.x

# Verify cluster connection (after starting Minikube/Kind/k3d)
kubectl version
# Should show both Client and Server versions
```

### Shell Completion

Enable tab completion for faster command entry:

```bash
# Bash
source <(kubectl completion bash)
echo "source <(kubectl completion bash)" >> ~/.bashrc

# Zsh
source <(kubectl completion zsh)
echo "source <(kubectl completion zsh)" >> ~/.zshrc

# Fish
kubectl completion fish | source
kubectl completion fish > ~/.config/fish/completions/kubectl.fish
```

## 13.4 Cluster Configuration

Kubernetes uses kubeconfig files to store cluster connection information, credentials, and contexts. Understanding this configuration is essential for managing multiple clusters.

### Kubeconfig Structure

The default location is `~/.kube/config` (or `%USERPROFILE%\.kube\config` on Windows). This YAML file contains three main sections:

```yaml
apiVersion: v1
kind: Config
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0t...
    server: https://192.168.49.2:8443
  name: minikube
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTi...
    server: https://0.0.0.0:12345
  name: kind-dev-cluster

users:
- name: minikube
  user:
    client-certificate-data: LS0tLS1CRUdJTi...
    client-key-data: LS0tLS1CRUdJTi...
- name: kind-dev-cluster
  user:
    client-certificate-data: LS0tLS1CRUdJTi...

contexts:
- context:
    cluster: minikube
    namespace: default
    user: minikube
  name: minikube
- context:
    cluster: kind-dev-cluster
    namespace: default
    user: kind-dev-cluster
  name: kind-dev-cluster

current-context: minikube
```

**Clusters**: Define API server endpoints and CA certificates  
**Users**: Define authentication credentials (certificates, tokens, or passwords)  
**Contexts**: Bind users to clusters and specify default namespaces  

### Managing Kubeconfig

**View current configuration:**
```bash
# View full config
kubectl config view

# View current context
kubectl config current-context

# List all contexts
kubectl config get-contexts

# Detailed view with certificates
kubectl config view --raw
```

**Switching contexts:**
```bash
# Switch to different cluster
kubectl config use-context kind-dev-cluster

# Set default namespace for context
kubectl config set-context --current --namespace=production

# Create new context entry
kubectl config set-context prod-admin \
  --cluster=prod-cluster \
  --user=admin \
  --namespace=default
```

**Environment Variables:**
You can specify alternative config files:

```bash
# Use specific config file
export KUBECONFIG=~/.kube/config:~/.kube/prod-config

# Merge configs (temporary)
kubectl config view --merge --flatten > ~/.kube/merged-config
export KUBECONFIG=~/.kube/merged-config
```

### Security Best Practices

1. **Restrict file permissions**: Kubeconfig contains sensitive credentials
   ```bash
   chmod 600 ~/.kube/config
   ```

2. **Avoid embedding certificates**: Use file references instead of base64 data for easier rotation
   ```yaml
   user:
     client-certificate: /path/to/cert.pem
     client-key: /path/to/key.pem
   ```

3. **Use short-lived tokens**: For production, integrate with OIDC or cloud IAM rather than static certificates

4. **Separate configs**: Maintain separate files for production and development to prevent accidental changes

## 13.5 Basic kubectl Commands

kubectl organizes commands into verbs (actions) and nouns (resource types). Understanding this structure accelerates proficiency.

### Resource Management

**Creating Resources:**
```bash
# Create from file
kubectl apply -f deployment.yaml

# Create from multiple files
kubectl apply -f ./manifests/

# Create from URL
kubectl apply -f https://raw.githubusercontent.com/org/repo/main/deployment.yaml

# Create imperatively (quick tests only)
kubectl create deployment nginx --image=nginx:1.25
kubectl create service clusterip nginx --tcp=80:80
```

**Viewing Resources:**
```bash
# List pods
kubectl get pods

# List with wide output (more columns)
kubectl get pods -o wide

# List all namespaces
kubectl get pods --all-namespaces
# or
kubectl get pods -A

# Detailed yaml output
kubectl get pod nginx -o yaml

# JSON output with specific fields
kubectl get pod nginx -o jsonpath='{.status.phase}'

# Watch for changes (real-time updates)
kubectl get pods -w

# Show labels
kubectl get pods --show-labels
```

**Describing Resources:**
```bash
# Detailed information including events
kubectl describe pod nginx

# Describe nodes for capacity and conditions
kubectl describe node minikube
```

**Deleting Resources:**
```bash
# Delete by file
kubectl delete -f deployment.yaml

# Delete by name
kubectl delete deployment nginx

# Delete by label selector
kubectl delete pods -l app=nginx

# Delete gracefully (default 30s timeout)
kubectl delete pod nginx --grace-period=30

# Force delete (ungraceful, use for stuck pods)
kubectl delete pod nginx --force
```

### Debugging Commands

**Pod Interaction:**
```bash
# Execute command in pod
kubectl exec nginx -- ls -la /etc/nginx

# Interactive shell
kubectl exec -it nginx -- /bin/bash
# (Use /bin/sh if bash unavailable)

# Copy files to/from pod
kubectl cp nginx:/etc/nginx/nginx.conf ./nginx.conf
kubectl cp ./local-file.txt nginx:/tmp/

# Port forward (access cluster service locally)
kubectl port-forward pod/nginx 8080:80
# Access via http://localhost:8080

# Port forward service
kubectl port-forward svc/nginx-service 8080:80
```

**Logs:**
```bash
# View logs
kubectl logs nginx

# Follow logs (tail -f equivalent)
kubectl logs -f nginx

# Previous container logs (after crash/restart)
kubectl logs nginx --previous

# Show timestamps
kubectl logs nginx --timestamps

# Tail last N lines
kubectl logs nginx --tail=100
```

**Cluster Debugging:**
```bash
# View node resource usage
kubectl top node

# View pod resource usage
kubectl top pod

# Check API server connectivity
kubectl cluster-info

# View cluster events (useful for troubleshooting)
kubectl get events --sort-by='.lastTimestamp'
kubectl get events --field-selector type=Warning
```

## 13.6 Understanding Namespaces

Namespaces provide a mechanism for isolating groups of resources within a single cluster. They are intended for use in environments with many users spread across multiple teams or projects.

### Namespace Concepts

**Scope of Names:**
Resource names need only be unique within a namespace. You can have a Pod named "web" in namespace "development" and another Pod named "web" in namespace "production".

**Default Namespaces:**
Kubernetes starts with four initial namespaces:
- **default**: For resources without a specified namespace
- **kube-system**: For Kubernetes system components (control plane, addons)
- **kube-public**: For publicly accessible data (configmaps with cluster info)
- **kube-node-lease**: For node heartbeat leases (lightweight availability detection)

### Working with Namespaces

**Creating Namespaces:**
```yaml
# namespace-dev.yaml
apiVersion: v1
kind: Namespace
metadata:
  name: development
  labels:
    environment: dev
    cost-center: engineering
```

```bash
kubectl apply -f namespace-dev.yaml

# Or imperatively
kubectl create namespace production
```

**Namespace Operations:**
```bash
# List namespaces
kubectl get namespaces

# List resources in specific namespace
kubectl get pods -n development

# Set default namespace for current context
kubectl config set-context --current --namespace=development

# View resources across all namespaces
kubectl get pods --all-namespaces

# Delete namespace (deletes all contained resources)
kubectl delete namespace development
```

### Namespace Use Cases

**Environment Separation:**
Rather than maintaining separate clusters for dev/staging/prod (expensive), use namespaces with resource quotas:

```yaml
apiVersion: v1
kind: ResourceQuota
metadata:
  name: dev-quota
  namespace: development
spec:
  hard:
    requests.cpu: "10"
    requests.memory: 20Gi
    limits.cpu: "20"
    limits.memory: 40Gi
    pods: "50"
    services: "20"
```

**Team Isolation:**
Different teams own different namespaces with Role-Based Access Control (RBAC) restricting access.

**Blue/Green Deployments:**
Deploy new versions in separate namespaces for instant switching:

```bash
kubectl apply -f app-v2.yaml -n blue
# Test, then switch service selector or ingress to blue namespace
```

### Networking and Namespaces

Namespaces provide DNS isolation. Services in different namespaces reference each other using fully qualified domain names (FQDN):

```bash
# From a pod in namespace 'development'
curl http://backend-service          # Same namespace
curl http://backend-service.production.svc.cluster.local  # Different namespace
# Format: <service-name>.<namespace>.svc.cluster.local
```

## 13.7 Context and Configuration

As you work with multiple clusters (local development, staging, production), kubectl contexts enable rapid switching without manual reconfiguration.

### Context Management Workflow

**Typical Setup:**
```bash
# Development (Minikube)
kubectl config set-cluster dev-cluster --server=https://192.168.49.2:8443 --certificate-authority=/path/to/ca.crt
kubectl config set-credentials dev-user --client-certificate=/path/to/dev.crt --client-key=/path/to/dev.key
kubectl config set-context dev --cluster=dev-cluster --user=dev-user --namespace=default

# Production (EKS)
aws eks update-kubeconfig --region us-west-2 --name prod-cluster
# This automatically adds context named arn:aws:eks:region:account:cluster/prod-cluster

# Switch between them
kubectl config use-context dev
kubectl config use-context arn:aws:eks:region:account:cluster/prod-cluster
```

**Visual Indicators:**
Consider adding current context to your shell prompt to prevent accidental production changes:

```bash
# Add to ~/.bashrc or ~/.zshrc
source <(kubectl completion bash)
alias k=kubectl
complete -o default -F __start_kubectl k

# Show current context in prompt (using kube-ps1)
source /usr/local/opt/kube-ps1/share/kube-ps1.sh
PS1='$(kube_ps1)'$PS1
```

### Safety Mechanisms

**Production Protection:**
Add visual warnings for production contexts:

```bash
# ~/.bashrc
function kubectl() {
    local context=$(kubectl config current-context 2>/dev/null)
    if [[ "$context" == *"prod"* ]] && [[ "$1" == "delete" ]]; then
        read -p "⚠️  PRODUCTION DELETE. Are you sure? (type 'yes'): " confirm
        if [[ "$confirm" != "yes" ]]; then
            echo "Aborted."
            return 1
        fi
    fi
    command kubectl "$@"
}
```

**Dry Runs:**
Always validate changes before applying:

```bash
# Client-side dry run (validation only)
kubectl apply -f deployment.yaml --dry-run=client

# Server-side dry run (validates against API server without persisting)
kubectl apply -f deployment.yaml --dry-run=server
```

## 13.8 First Deployment

We conclude this chapter by deploying a practical application: a multi-tier web application consisting of a frontend (nginx) and backend (simple HTTP API). This demonstrates namespace usage, service exposure, and resource management.

### Step 1: Create Namespace

```bash
kubectl create namespace first-app
kubectl config set-context --current --namespace=first-app
```

### Step 2: Deploy Backend

```yaml
# backend.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: backend
  labels:
    app: backend
spec:
  replicas: 2
  selector:
    matchLabels:
      app: backend
  template:
    metadata:
      labels:
        app: backend
    spec:
      containers:
      - name: backend
        image: kennethreitz/httpbin:latest
        ports:
        - containerPort: 80
        resources:
          requests:
            memory: "128Mi"
            cpu: "100m"
          limits:
            memory: "256Mi"
            cpu: "200m"
---
apiVersion: v1
kind: Service
metadata:
  name: backend
spec:
  selector:
    app: backend
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
  type: ClusterIP
```

Deploy:
```bash
kubectl apply -f backend.yaml
kubectl get pods -l app=backend
kubectl logs -l app=backend
```

### Step 3: Deploy Frontend

```yaml
# frontend.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: frontend
spec:
  replicas: 1
  selector:
    matchLabels:
      app: frontend
  template:
    metadata:
      labels:
        app: frontend
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        ports:
        - containerPort: 80
        volumeMounts:
        - name: config
          mountPath: /etc/nginx/conf.d
      volumes:
      - name: config
        configMap:
          name: nginx-config
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: nginx-config
data:
  default.conf: |
    server {
      listen 80;
      location / {
        proxy_pass http://backend:80;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
      }
    }
---
apiVersion: v1
kind: Service
metadata:
  name: frontend
spec:
  selector:
    app: frontend
  ports:
    - port: 80
      targetPort: 80
  type: NodePort  # Exposes on host port for local access
```

Deploy:
```bash
kubectl apply -f frontend.yaml
```

### Step 4: Access Application

```bash
# Get NodePort
kubectl get svc frontend
# Output shows port like 80:30080/TCP

# For Minikube
minikube service frontend --url

# For Kind (requires port mapping during cluster creation)
kubectl port-forward svc/frontend 8080:80
# Access http://localhost:8080

# Test backend connectivity through frontend
curl http://localhost:8080/get
```

### Step 5: Scale and Update

```bash
# Scale backend
kubectl scale deployment backend --replicas=3

# Watch scaling
kubectl get pods -w

# Update image (simulating new version)
kubectl set image deployment/backend backend=kennethreitz/httpbin:latest --record

# Check rollout status
kubectl rollout status deployment/backend

# View rollout history
kubectl rollout history deployment/backend

# Rollback if needed
kubectl rollout undo deployment/backend
```

### Cleanup

```bash
kubectl delete namespace first-app
# Or if using Minikube/Kind/k3d, simply delete the cluster
minikube delete
# or
kind delete cluster
# or
k3d cluster delete mycluster
```

---

## Chapter Summary and Preview

In this chapter, we transitioned from Kubernetes architecture theory to practical implementation. You installed and configured local Kubernetes clusters using Minikube (VM-based, feature-rich), Kind (Docker-native, CI/CD optimized), and k3d (ultra-lightweight, fast startup). We configured kubectl for cross-platform usage, explored kubeconfig file structures for multi-cluster management, and established namespace-based isolation patterns. Basic kubectl commands for CRUD operations, debugging, and log analysis provide the foundation for daily operations. Finally, you deployed a complete multi-tier application demonstrating service discovery, configuration management, and scaling workflows.

**Key Takeaways:**
- Local clusters (Minikube, Kind, k3d) provide production-like environments for development without cloud costs.
- kubectl uses kubeconfig files to manage cluster connections; contexts enable safe switching between environments.
- Namespaces isolate resources within a cluster, enabling environment separation and team boundaries without maintaining multiple clusters.
- Always use --dry-run=server before applying changes to production environments.
- Resource specifications (requests/limits) ensure applications coexist without resource starvation.

**Next Chapter Preview:**
Chapter 14: Kubernetes Core Resources delves into the fundamental building blocks of Kubernetes applications. You will master Pods in detail (lifecycle, probes, multi-container patterns), Deployments (rolling updates, rollback strategies, ReplicaSets), Services (ClusterIP, NodePort, LoadBalancer, Endpoints), ConfigMaps and Secrets (configuration management), and the label/selector system enabling loose coupling. These resources form the vocabulary through which you express application requirements to the Kubernetes control plane introduced in Chapter 12.